In [1]:
# VGG16 with finetuning

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

In [3]:
# load CIFAR-10 dataset
(train_images, train_labels), (test_images,test_labels) =   datasets.cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [4]:
len(train_images), len(train_labels), len(test_images), len(test_labels),

(50000, 50000, 10000, 10000)

In [5]:
#Normalize pixel values to be between 0 and 1

In [6]:
train_images, test_images = train_images/255.0, test_images/255.0

In [7]:
# Load pretrained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

58889256/58889256 [==============================] - 0s 0us/step


In [8]:
base_model

In [9]:
#Freeze convolutional layers
for layer in base_model.layers:
  layer.trainable = False

In [10]:
# Add customer classifier
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')

  ]
)

In [11]:
# Compile the model
model.compile(optimizer=Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Train the model
history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
1563/1563 [==============================] - 22s 11ms/step - loss: 1.4885 - accuracy: 0.4785 - val_loss: 1.2756 - val_accuracy: 0.5528
Epoch 2/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.2981 - accuracy: 0.5457 - val_loss: 1.2083 - val_accuracy: 0.5795
Epoch 3/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.2469 - accuracy: 0.5621 - val_loss: 1.1933 - val_accuracy: 0.5799
Epoch 4/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.2120 - accuracy: 0.5758 - val_loss: 1.1657 - val_accuracy: 0.5901
Epoch 5/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.1909 - accuracy: 0.5823 - val_loss: 1.1564 - val_accuracy: 0.5946
Epoch 6/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.1672 - accuracy: 0.5905 - val_loss: 1.1448 - val_accuracy: 0.6008
Epoch 7/10
1563/1563 [==============================] - 16s 10ms/step - loss: 1.1476 - accuracy: 0.5992 - val_loss: 1.1396 -

In [13]:
# Evlauate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 3s 9ms/step - loss: 1.1371 - accuracy: 0.6057


In [14]:
print(test_loss, test_acc)

1.1371339559555054 0.6057000160217285


In [15]:
# Fine-tuning


In [16]:
#unfreeze some layers for fine-tuning
for layer in base_model.layers[-4:]:
  layer.trainable = True

In [17]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
1563/1563 [==============================] - 30s 18ms/step - loss: 1.1004 - accuracy: 0.6238 - val_loss: 0.9864 - val_accuracy: 0.6666
Epoch 2/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.8388 - accuracy: 0.7124 - val_loss: 0.8708 - val_accuracy: 0.7040
Epoch 3/10
1563/1563 [==============================] - 26s 17ms/step - loss: 0.7118 - accuracy: 0.7566 - val_loss: 0.8240 - val_accuracy: 0.7339
Epoch 4/10
1563/1563 [==============================] - 26s 17ms/step - loss: 0.6086 - accuracy: 0.7870 - val_loss: 0.8169 - val_accuracy: 0.7296
Epoch 5/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.5066 - accuracy: 0.8229 - val_loss: 0.7757 - val_accuracy: 0.7479
Epoch 6/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.4216 - accuracy: 0.8515 - val_loss: 0.8583 - val_accuracy: 0.7523
Epoch 7/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.3478 - accuracy: 0.8757 - val_loss: 0.9088 -

In [19]:
# try using differnet learning rates
learning_rates = [0.0001, 0.0005, 0.001, 0.005]
results = {}

In [24]:
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')

  ]
)

In [21]:
for lr in learning_rates:
  model.compile(optimizer=Adam(learning_rate=lr),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
  # Store the results
  results[lr] = history.history['val_accuracy']

Epoch 1/10
1563/1563 [==============================] - 29s 18ms/step - loss: 0.3906 - accuracy: 0.8716 - val_loss: 0.9151 - val_accuracy: 0.7459
Epoch 2/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.1678 - accuracy: 0.9437 - val_loss: 0.9935 - val_accuracy: 0.7453
Epoch 3/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.1213 - accuracy: 0.9599 - val_loss: 1.1796 - val_accuracy: 0.7466
Epoch 4/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0938 - accuracy: 0.9688 - val_loss: 1.2319 - val_accuracy: 0.7452
Epoch 5/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0836 - accuracy: 0.9730 - val_loss: 1.2724 - val_accuracy: 0.7431
Epoch 6/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0760 - accuracy: 0.9753 - val_loss: 1.2840 - val_accuracy: 0.7503
Epoch 7/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.0662 - accuracy: 0.9792 - val_loss: 1.3189 -

In [22]:
for lr, val_acc in results.items():
    print(f"Learning rate: {lr}, Validation Accuracy: {max(val_acc)}")

Learning rate: 0.0001, Validation Accuracy: 0.7502999901771545
Learning rate: 0.0005, Validation Accuracy: 0.7418000102043152
Learning rate: 0.001, Validation Accuracy: 0.7268999814987183
Learning rate: 0.005, Validation Accuracy: 0.10000000149011612


In [25]:
model.compile(optimizer=Adam(learning_rate=0.00005),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))
# Store the results
results[lr] = history.history['val_accuracy']

Epoch 1/10
1563/1563 [==============================] - 29s 17ms/step - loss: 52.2658 - accuracy: 0.2621 - val_loss: 1.8951 - val_accuracy: 0.4404
Epoch 2/10
1563/1563 [==============================] - 27s 17ms/step - loss: 1.8407 - accuracy: 0.4286 - val_loss: 1.7486 - val_accuracy: 0.4737
Epoch 3/10
1563/1563 [==============================] - 26s 17ms/step - loss: 1.5278 - accuracy: 0.5121 - val_loss: 1.5864 - val_accuracy: 0.5380
Epoch 4/10
1563/1563 [==============================] - 26s 17ms/step - loss: 1.2994 - accuracy: 0.5869 - val_loss: 1.4582 - val_accuracy: 0.5798
Epoch 5/10
1563/1563 [==============================] - 27s 17ms/step - loss: 1.1033 - accuracy: 0.6496 - val_loss: 1.3774 - val_accuracy: 0.6364
Epoch 6/10
1563/1563 [==============================] - 26s 17ms/step - loss: 0.9255 - accuracy: 0.7048 - val_loss: 1.3466 - val_accuracy: 0.6649
Epoch 7/10
1563/1563 [==============================] - 27s 17ms/step - loss: 0.7810 - accuracy: 0.7519 - val_loss: 1.3264 